# Utilizing `chempy` to balance chemical equations

`chempy` is already so powerful that it is able to balance a skeletal equation with just a one-line execution of their code. This notebook is focused on using thermodynamic data to predict the products of a reaction if only the reactants are known.

In [3]:
import os                               
import re                   
import time                 # to stall requests (just in case)
import itertools
import sympy
import pickle

import numpy as np
import pandas as pd 

from tika import parser     # the specific parser method 

from chempy import balance_stoichiometry
from chempy import Substance
from chempy import Reaction
from chempy.util import periodic

pd.set_option('display.max_colwidth', 0)    # no max column width
pd.set_option('display.max_rows', 1000)

In [4]:
stoich_df = pickle.load(open('../data/processed/stoich_df.p', 'rb'))
thermo_df = pickle.load(open('../data/processed/thermo_df.p', 'rb'))

In [5]:
stoich_df['formula']

0       C4H9SH(aq)         
1       C10H14N5O10P2-(aq) 
2       C5H12S2(l)         
3       C7H12(aq)          
4       C6H10N3O2+(aq)     
             ...           
2825    C12H26(l)          
2826    AsS(OH)HS-(aq)     
2827    NaC4H4O5-(aq)      
2828    Eu(CH3CH2CO2)2+(aq)
2829    C9H12N2O5(aq)      
Name: formula, Length: 2830, dtype: object

In [6]:
new_columns = []
for col in stoich_df.columns:
    try:
        new_columns.append(int(col))
    except:
        new_columns.append(col)
new_columns

['formula',
 0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 74,
 75,
 78,
 79,
 80,
 81,
 82,
 83,
 86,
 87,
 88,
 90,
 91,
 92]

In [7]:
stoich_df.columns = new_columns

## playground for writing functions using thermo tables

The cells below show my thought process exploring how to relate the `chempy` library with the thermodynamic tables. The goals are as follows:

- The most likely chemical reaction is highly correlated with the reaction that has the greatest loss of Gibbs free energy (given by parameter 'G' in `thermo`.
- We iterate through the different possible products so long as their combination allows for balanced stoichiometry (having the same number of each element on both sides of the equation)

We will attempt to predict the following reaction:

$$ 2 Na(s) + 2 H_{2}O(l) \longrightarrow 2 NaOH(aq) + H_{2}(g) $$

In [8]:
reactants = ['Na', 'H2O']

In [9]:
water = Substance.from_formula('H2O')
[*water.composition]

[1, 8]

When searching for possible products, we want to ignore all compounds that have elements outside of sodium, hydrogen, or oxygen.

In [10]:
z_ignore = ['formula']
for r in reactants:
    s = Substance.from_formula(r)
    z_ignore += [*s.composition]
z_ignore = set(z_ignore)
z_ignore

{1, 11, 8, 'formula'}

In [11]:
column_mask = [col for col in stoich_df.columns if col not in z_ignore]
print(column_mask)

[0, 2, 3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 74, 75, 78, 79, 80, 81, 82, 83, 86, 87, 88, 90, 91, 92]


In [12]:
# https://stackoverflow.com/questions/22649693/

stoich_temp = stoich_df.copy()

for col in column_mask:
    stoich_temp = stoich_temp[stoich_temp[col] == 0]

stoich_temp = stoich_temp.loc[(stoich_temp.drop(columns=['formula'])!=0).any(axis=1)]

stoich_temp = stoich_temp[z_ignore]
stoich_temp

,8,1,11,formula
11,1.0,2.0,0.0,"H2O(s, VIII)"
152,1.0,0.0,0.0,O(g)
266,3.0,0.0,0.0,O3(g)
341,1.0,1.0,1.0,NaOH(aq)
586,0.0,0.0,1.0,Na(s)
672,1.0,2.0,0.0,"H2O(s, IX)"
782,2.0,2.0,0.0,H2O2(aq)
866,2.0,2.0,0.0,H2O2(l)
987,1.0,2.0,0.0,"H2O(s, III)"
1462,1.0,1.0,1.0,NaOH(s)


In [13]:
def formula_state_separator(formula, keep_state=False):
    '''
    Separates the state from a formula string.
    
    --Parameters--
    formula:        str
        a string of a single chemical formula
    
    --Output--
    tuple (str)
        
    --Examples--
    >>> formula_state_separator('NaCl(aq)')
    ('NaCl', 'aq')
    
    >>> formula_state_separator('NaCl')
    'NaCl'
    '''
    try:
        regex = re.search('(?<=\()[aglsq]+', formula)
        formula = formula[:regex.start() - 1]
        if keep_state:
            state = regex.group(0)
            return formula, state
        else:
            return formula
    except:
        return formula

In [14]:
# chemical reactions don't have the same species on both sides of the arrow

candidates = set([formula_state_separator(f) for f in set(stoich_temp['formula'])\
              if formula_state_separator(f) not in map(formula_state_separator, reactants)])
candidates

{'H', 'H2', 'H2O2', 'Na2O', 'NaOH', 'O', 'O2', 'O3'}

In [15]:
# most chemical reactions don't form more than four different chemical species
# we'll play it safe and just make the maximum number 3 + num_reactants

combinations = []
max_length = min(len(reactants) + 4, 7)
for i in range(1, max_length):
    combinations += list(itertools.combinations(candidates, i))
combinations

[('NaOH',),
 ('H2O2',),
 ('O3',),
 ('H',),
 ('H2',),
 ('Na2O',),
 ('O',),
 ('O2',),
 ('NaOH', 'H2O2'),
 ('NaOH', 'O3'),
 ('NaOH', 'H'),
 ('NaOH', 'H2'),
 ('NaOH', 'Na2O'),
 ('NaOH', 'O'),
 ('NaOH', 'O2'),
 ('H2O2', 'O3'),
 ('H2O2', 'H'),
 ('H2O2', 'H2'),
 ('H2O2', 'Na2O'),
 ('H2O2', 'O'),
 ('H2O2', 'O2'),
 ('O3', 'H'),
 ('O3', 'H2'),
 ('O3', 'Na2O'),
 ('O3', 'O'),
 ('O3', 'O2'),
 ('H', 'H2'),
 ('H', 'Na2O'),
 ('H', 'O'),
 ('H', 'O2'),
 ('H2', 'Na2O'),
 ('H2', 'O'),
 ('H2', 'O2'),
 ('Na2O', 'O'),
 ('Na2O', 'O2'),
 ('O', 'O2'),
 ('NaOH', 'H2O2', 'O3'),
 ('NaOH', 'H2O2', 'H'),
 ('NaOH', 'H2O2', 'H2'),
 ('NaOH', 'H2O2', 'Na2O'),
 ('NaOH', 'H2O2', 'O'),
 ('NaOH', 'H2O2', 'O2'),
 ('NaOH', 'O3', 'H'),
 ('NaOH', 'O3', 'H2'),
 ('NaOH', 'O3', 'Na2O'),
 ('NaOH', 'O3', 'O'),
 ('NaOH', 'O3', 'O2'),
 ('NaOH', 'H', 'H2'),
 ('NaOH', 'H', 'Na2O'),
 ('NaOH', 'H', 'O'),
 ('NaOH', 'H', 'O2'),
 ('NaOH', 'H2', 'Na2O'),
 ('NaOH', 'H2', 'O'),
 ('NaOH', 'H2', 'O2'),
 ('NaOH', 'Na2O', 'O'),
 ('NaOH', 'Na2O', 'O

Let's see if any of these combinations result in a good balanced equation:

In [16]:
for comb in combinations:    
    try:
        print(balance_stoichiometry(reactants, comb))
    except:
        pass

(OrderedDict([('Na', -6), ('H2O', -3)]), OrderedDict([('NaOH', -6), ('O3', 1)]))
(OrderedDict([('Na', 1), ('H2O', 1)]), OrderedDict([('NaOH', 1), ('H', 1)]))
(OrderedDict([('Na', 2), ('H2O', 2)]), OrderedDict([('NaOH', 2), ('H2', 1)]))
(OrderedDict([('Na', -2), ('H2O', -1)]), OrderedDict([('NaOH', -2), ('O', 1)]))
(OrderedDict([('Na', -4), ('H2O', -2)]), OrderedDict([('NaOH', -4), ('O2', 1)]))
(OrderedDict([('Na', 2), ('H2O', -1)]), OrderedDict([('H2O2', -1), ('Na2O', 1)]))
(OrderedDict([('Na', 2), ('H2O', 1)]), OrderedDict([('H', 2), ('Na2O', 1)]))
(OrderedDict([('Na', 2), ('H2O', 1)]), OrderedDict([('H2', 1), ('Na2O', 1)]))
(OrderedDict([('Na', -6*x1 - 2), ('H2O', -3*x1)]), OrderedDict([('NaOH', -6*x1 - 2), ('H2O2', 1), ('O3', x1)]))
(OrderedDict([('Na', x1 - 2), ('H2O', x1)]), OrderedDict([('NaOH', x1 - 2), ('H2O2', 1), ('H', x1)]))
(OrderedDict([('Na', 2*x1 - 2), ('H2O', 2*x1)]), OrderedDict([('NaOH', 2*x1 - 2), ('H2O2', 1), ('H2', x1)]))
(OrderedDict([('Na', -2), ('H2O', -x1)]), O

(OrderedDict([('Na', 2 - 4*x1), ('H2O', -2*x1 - x2 + 2)]), OrderedDict([('NaOH', -4*x1 - 2*x2 + 2), ('H2', 1), ('Na2O', x2), ('O2', x1)]))
(OrderedDict([('Na', -4*x1 - 2*x2 + 2), ('H2O', -2*x1 - x2 + 2)]), OrderedDict([('NaOH', -4*x1 - 2*x2 + 2), ('H2', 1), ('O', x2), ('O2', x1)]))
(OrderedDict([('Na', -4*x1 - 2*x2), ('H2O', -2*x1 - x2 - 1)]), OrderedDict([('NaOH', -4*x1 - 2*x2 - 2), ('Na2O', 1), ('O', x2), ('O2', x1)]))
(OrderedDict([('Na', 2*x1), ('H2O', -x1 + 2*x2 - 3)]), OrderedDict([('H2O2', -x1 + x2 - 3), ('O3', 1), ('H', 2*x2), ('Na2O', x1)]))
(OrderedDict([('Na', 2*x1), ('H2O', -x1 + 2*x2 - 3)]), OrderedDict([('H2O2', -x1 + x2 - 3), ('O3', 1), ('H2', x2), ('Na2O', x1)]))
(OrderedDict([('Na', 2*x2), ('H2O', -x1 - x2 - 3)]), OrderedDict([('H2O2', -x1 - x2 - 3), ('O3', 1), ('Na2O', x2), ('O', x1)]))
(OrderedDict([('Na', 2*x2), ('H2O', -2*x1 - x2 - 3)]), OrderedDict([('H2O2', -2*x1 - x2 - 3), ('O3', 1), ('Na2O', x2), ('O2', x1)]))
(OrderedDict([('Na', 2*x1), ('H2O', -x1 + 2*x2 + 1)

(OrderedDict([('Na', -4*x1 - 2*x2 + 2), ('H2O', -2*x1 - x2 - x3 + 2)]), OrderedDict([('NaOH', -4*x1 - 2*x2 - 2*x3 + 2), ('H2', 1), ('Na2O', x3), ('O', x2), ('O2', x1)]))
(OrderedDict([('Na', 2*x1), ('H2O', -x1 + 2*x2 + 2*x3 - 3)]), OrderedDict([('H2O2', -x1 + x2 + x3 - 3), ('O3', 1), ('H', 2*x3), ('H2', x2), ('Na2O', x1)]))
(OrderedDict([('Na', 2*x2), ('H2O', -x1 - x2 + 2*x3 - 3)]), OrderedDict([('H2O2', -x1 - x2 + x3 - 3), ('O3', 1), ('H', 2*x3), ('Na2O', x2), ('O', x1)]))
(OrderedDict([('Na', 2*x2), ('H2O', -2*x1 - x2 + 2*x3 - 3)]), OrderedDict([('H2O2', -2*x1 - x2 + x3 - 3), ('O3', 1), ('H', 2*x3), ('Na2O', x2), ('O2', x1)]))
(OrderedDict([('Na', 2*x2), ('H2O', -x1 - x2 + 2*x3 - 3)]), OrderedDict([('H2O2', -x1 - x2 + x3 - 3), ('O3', 1), ('H2', x3), ('Na2O', x2), ('O', x1)]))
(OrderedDict([('Na', 2*x2), ('H2O', -2*x1 - x2 + 2*x3 - 3)]), OrderedDict([('H2O2', -2*x1 - x2 + x3 - 3), ('O3', 1), ('H2', x3), ('Na2O', x2), ('O2', x1)]))
(OrderedDict([('Na', 2*x3), ('H2O', -2*x1 - x2 - x3 - 

In order for an equation to be properly balanced, each coefficient (dictionary values here) must be a positive number, and oftentimes we balance so that every coefficient is a whole number. We have to filter out the balanced instances where we get negative coefficients (such as the equation with `H2O2` and `Na2O` as products).

Notice also that `sympy` has relative coefficients listed (meaning the equation would be balanced for any whole number `x1`, for example). We will want to filter these out too, but may consider including them for a reach goal.

In [17]:
np.array(list(balance_stoichiometry(reactants, ('H2O2', 'Na2O'))[0].values())) >= 1

array([ True, False])

In [18]:
# there might be a more elegant way of doing this. round down to zero if any instance is false.

np.floor((np.array(list(balance_stoichiometry(reactants, ('H2O2', 'Na2O'))[0].values())) >= 1).mean())

0.0

In [19]:
np.array(list(balance_stoichiometry(reactants, ('H2', 'O2', 'NaOH'))[0].values()))

array([2*x1, 2*x1 + 4], dtype=object)

In [20]:
np.array([isinstance(i, sympy.numbers.Number) for i in list(balance_stoichiometry(reactants, ('H2', 'O2', 'NaOH'))[0].values())])

array([False, False])

## functions

The above code has been condensed into several functions:

- `formula_state_separator` (already defined): in the case that a formula is formatted with its corresponding state (eg: `NaCl(s)`), return just the formula.
- `get_gibbs`: from the results of `formula_state_separator`, find the exact free energy value for the substance specified. If state is not specified, find the lowest free energy value for formulas that have multiple entries (since that is the most likely state under standard conditions.
- `state_predictor`: from the results of `get_gibbs`, return the form of the compound with the lowest free energy value (therefore the most stable under standard conditions).
- `Z_unique`: returns a list of unique atomic numbers present in a list of substances
- `stoich_filter`: filters the `stoich` dataframe to return formulas that only have the elements described by `Z_unique`
- `check_coefficients`: checks if all coefficients are positive and non-relational once an equation has been balanced
- `possibility_reducer`: sometimes we get too many results from `stoich_filter`. In general, substances with lower $\Delta G$ values are more likely to be products. However, very large, complex molecules with low $\Delta G$ values are still not very likely, so one (imperfect) way to normalize for that is to divide by the mass of the compound. The jury is still out if this is a good way to filter.
- `standard_gibbs_free_energy`: calculates the overall $\Delta G$ change under standard conditions.
- `reaction_predictor`: takes a list of reactants, iterates through the different possibilities (using `stoich_filter`, takes valid combinations using `check_coefficients`, and calculates $\Delta G$ values using `thermo`. Returns the reaction with the lowest $\Delta G$ value.

In [21]:
def get_gibbs(formula, energy='G', df=False):
    '''
    Retrieves the free energy value, in J, of a single substance
    
    --Parameters--
    formula:        str
        a string of a single chemical formula
    
    --Output--
    list (float)    
        
    --Examples--
    >>> get_gibbs('NaCl(aq)')
    array([-388735.44])
    '''
    if (thermo_df['formula'] == formula).max():
        matches = thermo_df[thermo_df['formula'] == formula]
    else:
        matches = thermo_df[thermo_df['formula'].map(
            lambda x: x[:len(formula)] == formula)]
        matches = matches[matches['formula'].map(
            formula_state_separator) == formula]

    if df:
        return matches
    else:
        return list(matches[energy])[0]

In [22]:
get_gibbs('HNO3', df=True)

,formula,abbrv,name,G,H,S,Cp,mass
432,HNO3(aq),HNO3,HNO3,-103470.32,-189995.44,178.6568,75.312,63.012
2744,HNO3(l),HNO3,NaN,-80760.00,-174100.00,155.4900,109.870,63.012


In [23]:
stuff = ['Na', 'H2O', 'NaOH', 'H2']

In [24]:
stuff[::-1]

['H2', 'NaOH', 'H2O', 'Na']

In [25]:
for s in stuff:
    print(get_gibbs(s, 'S'))

51.21
44.772983999999994
44.7688
57.739200000000004


In [26]:
def state_predictor(formula):
    df = get_gibbs(formula, df=True)
    return list(df.sort_values(by='G')['formula'])[0]

In [27]:
state_predictor('HNO3')

'HNO3(aq)'

In [28]:
def Z_unique(substances):
    '''
    Returns a set representing unique atomic numbers present within a list of
    chemical formulas.
    
    --Parameters--
    substances:     iterable (str)
        any iterable containing strings with valid chemical formulas
    
    --Output--
    set (int)
        atomic numbers of each unique element present in substances
        
    --Example--
    >>> Z_unique(['CH4', 'H2O'])
    {1, 6, 8}
    '''
    if type(substances) == str:
        substances = [substances]
        
    composition = []
    for s in substances:
        sub = Substance.from_formula(s)
        composition += [*sub.composition]
    return set(composition)

In [29]:
reactants

['Na', 'H2O']

In [30]:
Z_unique(reactants)

{1, 8, 11}

In [31]:
def stoich_filter(substances, df=False, thorough=False):
    '''
    Returns a masked copy of the stoich dataframe containing elements that
    only contain the elements present in substances. 
    
    --Parameters--
    substances:     iterable (str)
        any iterable containing strings with valid chemical formulas
    
    --Output--
    DataFrame
    '''
    if type(substances) == str:
        substances = [substances]
    
    stoich_temp = stoich_df.copy()
    
    # mask to keep the charge and formula columns in final dataframe
    z_keep = [0, 'formula'] + list(Z_unique(substances))
    
    # get all other columns
    column_mask = [col for col in stoich_temp.columns if col not in z_keep]
    for col in column_mask:
        # return the dataframe where these columns are all 0
        stoich_temp = stoich_temp[stoich_temp[col] == 0]
    
    # keep the columns where it's not all zero
    stoich_temp = stoich_temp.loc[(stoich_temp.drop(columns=['formula'])!=0).any(axis=1)]
    
    # return the dataframe with the columns we want to keep
    if df:
        return stoich_temp[z_keep]
    else:
        stoich_list = list(stoich_temp['formula'])
        if thorough:
            return set([f for f in stoich_list if f not in substances])
        else:
            stoich_list = [formula_state_separator(f) for f in stoich_list]
            substances = [formula_state_separator(s) for s in substances]
            return set([state_predictor(f) for f in stoich_list if f not in substances])

In [32]:
stoich_filter(reactants)

{'H(g)',
 'H+(aq)',
 'H2(g)',
 'H2O2(aq)',
 'H3O+(aq)',
 'HO2-(aq)',
 'Na+(aq)',
 'Na2O(s)',
 'NaOH(aq)',
 'O(g)',
 'O2(g)',
 'O3(g)',
 'OH-(aq)',
 'e-(aq)'}

In [33]:
def check_coefficients(reactants, products):
    '''
    Checks whether a possible reactant/product combination would result in a
    valid balanced chemical equation.
    
    --Parameters--
    reactants:      iterable (str)
    products:       iterable (str)
        any iterable containing strings with valid chemical formulas
    
    --Output--
    bool
        
    --Examples--
    >>> check_coefficients(['CH4', 'H2O'], ['CO', 'H2'])
    True
    
    >>> check_coefficients(['CH4', 'H2O'], ['CO2', 'H2O2'])
    False
    
    >>> check_coefficients(['CH4', 'H2O'], ['NaOH'])
    False
    '''
    try:
        balance = balance_stoichiometry(reactants, products)
        
        # list all the coefficients out
        reac_coef = list(balance[0].values()) + list(balance[1].values())
        
        # rounds to zero if any of the coefficients are less than 1
        is_positive = np.floor((np.array(reac_coef) >= 1).mean()).astype(bool)
        
        # rounds to zero if any of the coefficients are sympy relational class
        is_definite = np.floor(np.array([isinstance(i, sympy.numbers.Number) for i in reac_coef])\
                               .mean()).astype(bool)
        
        return is_positive and is_definite
    except:
        return False   

In [34]:
check_coefficients(reactants, ['Na2O', 'H2O2'])

False

In [35]:
# https://stackoverflow.com/questions/6618515/

def possibility_reducer(possibilities, length=12, offset=0):
    '''
    Limits the list of possible substances to a specified length based on 
    free energy 'density'
    
    --Parameters--
    possibilities:      iterable (str)
        any iterable containing strings with valid chemical formulas
    
    --Output--
    tuple (str)
        
    --Examples--

    '''
    # just in case. might be redundant
    possibilities = np.array(list(possibilities))
    energies = np.array([min(get_gibbs(s)) / Substance.from_formula(s).mass for s in possibilities])
    indices = energies.argsort()
    sorted_possibilities = possibilities[indices]
    
    max_length = min(len(sorted_possibilities), (length + offset))
    
    return sorted_possibilities[offset:(max_length)]

In [49]:
def standard_gibbs_free_energy(reactants, products, energy='G', kJ=True):
    '''
    Returns the overall delG of a reaction under standard conditions. 
    
    --Parameters--
    reactants:      iterable (str)
    products:       iterable (str)
    
    --Output--
    float
        
    --Examples--
    >>> standard_gibbs_free_energy(['Na', 'H2O'], ['NaH', 'O2'])
    340.36
    '''
    products = [state_predictor(p) for p in products]
    reactants = [state_predictor(r) for r in reactants]
    equation = balance_stoichiometry(reactants, products)
    # each side is a formula, coefficient tuple
    prod = list(equation[1].items())
    reac = list(equation[0].items())
    delG = 0
    # s[0] is the formula, with or without state
    # s[1] is the coefficient

    def gibbs_sum(side):
        interim_delG = 0
        
        for s in side:
            interim_delG += get_gibbs(s[0], energy) * s[1]
        return interim_delG

    delG = gibbs_sum(prod) - gibbs_sum(reac)
    return delG / (1 + 999*kJ)

In [37]:
standard_gibbs_free_energy(['Na', 'H2O'], ['NaOH', 'H2'])

-361.603200000000

In [72]:
def reaction_predictor(reactants, max_length=12):
    '''
    Returns the balanced chemical equation of the predicted reaction based on
    minimizing overall delG values.
    
    --Parameters--
    reactants:      iterable(str)
        any iterable containing strings with valid chemical formulas
    
    --Output--
    chempy.chemistry.Reaction
        
    --Examples--
    >>> reaction_predictor(['Al', 'O2'])
    4 Al + 3 O2 → 2 Al2O3
    '''
    reactants = [state_predictor(r) for r in reactants]
    possibilities = stoich_filter(reactants)
    print('scoping possibilities...')
    if len(possibilities) > max_length:
        possibilities = np.array(list(possibilities))
        energies = np.array([get_gibbs(s, 'G') / get_gibbs(s, 'mass') ** 3 for s in possibilities])
        indices = energies.argsort()
        sorted_possibilities = possibilities[indices]
        possibilities = sorted_possibilities[:(max_length)]

    print('  optimizing combinations...')
    combinations = []
    print(possibilities)
    comb_length = min(6, len(reactants) + 3)
    for i in range(1, comb_length):
        combinations += list(itertools.combinations(possibilities, i))
    combinations = [c for c in combinations if Z_unique(c) == Z_unique(reactants)]
    print(combinations)

    print('    deriving equations...')
    good_combinations = []
    for i, comb in enumerate(combinations):
        if check_coefficients(reactants, comb):
            good_combinations.append(comb)

    print('      calculating energies...')
    energies = []
    for gc in good_combinations:
        energies.append(standard_gibbs_free_energy(reactants, gc))

    best_energy = min(energies)
    best_index = energies.index(best_energy)
    best_reaction = Reaction(*balance_stoichiometry(
        reactants, good_combinations[best_index]))

    print(best_reaction)
    print(f'delG = {best_energy:.4} kJ mol-1')

    return best_reaction

In [39]:
#     possibilities = np.array(list(possibilities))
#     energies = np.array([min(get_gibbs(s)) / Substance.from_formula(s).mass for s in possibilities])
#     indices = energies.argsort()
#     sorted_possibilities = possibilities[indices]
    
#     max_length = min(len(sorted_possibilities), (length + offset))
    
#     return sorted_possibilities[offset:(max_length)]

In [69]:
reaction_predictor(['Na', 'H2O'])

scoping possibilities...
  optimizing combinations...
['H3O+(aq)' 'OH-(aq)' 'Na+(aq)' 'NaOH(aq)' 'H2O2(aq)' 'Na2O(s)' 'HO2-(aq)'
 'H+(aq)' 'H2(g)' 'e-(aq)' 'O2(g)' 'O3(g)']
[('NaOH(aq)',), ('NaOH(aq)', 'H2O2(aq)'), ('NaOH(aq)', 'Na2O(s)'), ('NaOH(aq)', 'H2(g)'), ('NaOH(aq)', 'O2(g)'), ('NaOH(aq)', 'O3(g)'), ('H2O2(aq)', 'Na2O(s)'), ('Na2O(s)', 'H2(g)'), ('NaOH(aq)', 'H2O2(aq)', 'Na2O(s)'), ('NaOH(aq)', 'H2O2(aq)', 'H2(g)'), ('NaOH(aq)', 'H2O2(aq)', 'O2(g)'), ('NaOH(aq)', 'H2O2(aq)', 'O3(g)'), ('NaOH(aq)', 'Na2O(s)', 'H2(g)'), ('NaOH(aq)', 'Na2O(s)', 'O2(g)'), ('NaOH(aq)', 'Na2O(s)', 'O3(g)'), ('NaOH(aq)', 'H2(g)', 'O2(g)'), ('NaOH(aq)', 'H2(g)', 'O3(g)'), ('NaOH(aq)', 'O2(g)', 'O3(g)'), ('H2O2(aq)', 'Na2O(s)', 'H2(g)'), ('H2O2(aq)', 'Na2O(s)', 'O2(g)'), ('H2O2(aq)', 'Na2O(s)', 'O3(g)'), ('Na2O(s)', 'H2(g)', 'O2(g)'), ('Na2O(s)', 'H2(g)', 'O3(g)'), ('NaOH(aq)', 'H2O2(aq)', 'Na2O(s)', 'H2(g)'), ('NaOH(aq)', 'H2O2(aq)', 'Na2O(s)', 'O2(g)'), ('NaOH(aq)', 'H2O2(aq)', 'Na2O(s)', 'O3(g)'), ('

In [67]:
reaction_predictor(['Al', 'O2'])

scoping possibilities...
  optimizing combinations...
{'Al+3(aq)', 'Al2O3(s)', 'O3(g)', 'O(g)', 'e-(aq)'}
[('Al2O3(s)',), ('Al2O3(s)', 'O3(g)'), ('Al2O3(s)', 'O(g)'), ('Al2O3(s)', 'O3(g)', 'O(g)')]
    deriving equations...
      calculating energies...
4 Al(s) + 3 O2(g) -> 2 Al2O3(s)
delG = -3165 kJ mol-1


In [68]:
reaction_predictor(['HCl(aq)', 'NaOH(s)'])

scoping possibilities...
  optimizing combinations...
['H2O(l)' 'H3O+(aq)' 'OH-(aq)' 'Na+(aq)' 'H2O2(aq)' 'NaCl(aq)' 'Cl-(aq)'
 'Na2O(s)' 'HO2-(aq)' 'HClO(aq)' 'ClO-(aq)' 'ClO-4(aq)']
[('H2O(l)', 'NaCl(aq)'), ('H2O2(aq)', 'NaCl(aq)'), ('NaCl(aq)', 'HClO(aq)'), ('Na2O(s)', 'HClO(aq)'), ('H2O(l)', 'H2O2(aq)', 'NaCl(aq)'), ('H2O(l)', 'NaCl(aq)', 'Na2O(s)'), ('H2O(l)', 'NaCl(aq)', 'HClO(aq)'), ('H2O(l)', 'Na2O(s)', 'HClO(aq)'), ('H2O2(aq)', 'NaCl(aq)', 'Na2O(s)'), ('H2O2(aq)', 'NaCl(aq)', 'HClO(aq)'), ('H2O2(aq)', 'Na2O(s)', 'HClO(aq)'), ('NaCl(aq)', 'Na2O(s)', 'HClO(aq)'), ('H2O(l)', 'H2O2(aq)', 'NaCl(aq)', 'Na2O(s)'), ('H2O(l)', 'H2O2(aq)', 'NaCl(aq)', 'HClO(aq)'), ('H2O(l)', 'H2O2(aq)', 'Na2O(s)', 'HClO(aq)'), ('H2O(l)', 'NaCl(aq)', 'Na2O(s)', 'HClO(aq)'), ('H2O2(aq)', 'NaCl(aq)', 'Na2O(s)', 'HClO(aq)')]
    deriving equations...
      calculating energies...
HCl(aq) + NaOH(s) -> H2O(l) + NaCl(aq)
delG = -119.1 kJ mol-1


In [64]:
reaction_predictor(['H2CO3'])

scoping possibilities...
  optimizing combinations...
['H2O(l)' 'HCO-3(aq)' 'H3O+(aq)' 'OH-(aq)' 'HC2O-4(aq)' 'CO2(g)' 'CH4(g)'
 'H2CO2(aq)' 'HCOOH(aq)' 'CO(g)' 'COOH-(aq)' 'HCO2-(aq)']
[('H2CO2(aq)',), ('HCOOH(aq)',), ('H2O(l)', 'CO2(g)'), ('H2O(l)', 'CH4(g)'), ('H2O(l)', 'H2CO2(aq)'), ('H2O(l)', 'HCOOH(aq)'), ('H2O(l)', 'CO(g)'), ('CO2(g)', 'CH4(g)'), ('CO2(g)', 'H2CO2(aq)'), ('CO2(g)', 'HCOOH(aq)'), ('CH4(g)', 'H2CO2(aq)'), ('CH4(g)', 'HCOOH(aq)'), ('CH4(g)', 'CO(g)'), ('H2CO2(aq)', 'HCOOH(aq)'), ('H2CO2(aq)', 'CO(g)'), ('HCOOH(aq)', 'CO(g)'), ('H2O(l)', 'CO2(g)', 'CH4(g)'), ('H2O(l)', 'CO2(g)', 'H2CO2(aq)'), ('H2O(l)', 'CO2(g)', 'HCOOH(aq)'), ('H2O(l)', 'CO2(g)', 'CO(g)'), ('H2O(l)', 'CH4(g)', 'H2CO2(aq)'), ('H2O(l)', 'CH4(g)', 'HCOOH(aq)'), ('H2O(l)', 'CH4(g)', 'CO(g)'), ('H2O(l)', 'H2CO2(aq)', 'HCOOH(aq)'), ('H2O(l)', 'H2CO2(aq)', 'CO(g)'), ('H2O(l)', 'HCOOH(aq)', 'CO(g)'), ('CO2(g)', 'CH4(g)', 'H2CO2(aq)'), ('CO2(g)', 'CH4(g)', 'HCOOH(aq)'), ('CO2(g)', 'CH4(g)', 'CO(g)'), ('CO2(

In [66]:
reaction_predictor(['Cu+2(aq)', 'Zn(s)'])

scoping possibilities...
  optimizing combinations...
{'e-(aq)', 'Cu(s)', 'Cu+(aq)', 'Zn+2(aq)'}
[('Cu(s)', 'Zn+2(aq)'), ('Cu+(aq)', 'Zn+2(aq)'), ('e-(aq)', 'Cu(s)', 'Zn+2(aq)'), ('e-(aq)', 'Cu+(aq)', 'Zn+2(aq)'), ('Cu(s)', 'Cu+(aq)', 'Zn+2(aq)'), ('e-(aq)', 'Cu(s)', 'Cu+(aq)', 'Zn+2(aq)')]
    deriving equations...
      calculating energies...
Cu+2(aq) + Zn(s) -> Cu(s) + Zn+2(aq)
delG = -212.6 kJ mol-1


In [73]:
reaction_predictor(['CH4', 'O2'])

scoping possibilities...
  optimizing combinations...
['H2O(l)' 'H3O+(aq)' 'OH-(aq)' 'HCO-3(aq)' 'HC2O-4(aq)' 'CO(g)'
 'CH3OH(aq)' 'CO2(g)' 'HCHO(aq)' 'COOH-(aq)' 'HCO2-(aq)' 'H2CO2(aq)']
[('CH3OH(aq)',), ('HCHO(aq)',), ('H2CO2(aq)',), ('H2O(l)', 'CO(g)'), ('H2O(l)', 'CH3OH(aq)'), ('H2O(l)', 'CO2(g)'), ('H2O(l)', 'HCHO(aq)'), ('H2O(l)', 'H2CO2(aq)'), ('CO(g)', 'CH3OH(aq)'), ('CO(g)', 'HCHO(aq)'), ('CO(g)', 'H2CO2(aq)'), ('CH3OH(aq)', 'CO2(g)'), ('CH3OH(aq)', 'HCHO(aq)'), ('CH3OH(aq)', 'H2CO2(aq)'), ('CO2(g)', 'HCHO(aq)'), ('CO2(g)', 'H2CO2(aq)'), ('HCHO(aq)', 'H2CO2(aq)'), ('H2O(l)', 'CO(g)', 'CH3OH(aq)'), ('H2O(l)', 'CO(g)', 'CO2(g)'), ('H2O(l)', 'CO(g)', 'HCHO(aq)'), ('H2O(l)', 'CO(g)', 'H2CO2(aq)'), ('H2O(l)', 'CH3OH(aq)', 'CO2(g)'), ('H2O(l)', 'CH3OH(aq)', 'HCHO(aq)'), ('H2O(l)', 'CH3OH(aq)', 'H2CO2(aq)'), ('H2O(l)', 'CO2(g)', 'HCHO(aq)'), ('H2O(l)', 'CO2(g)', 'H2CO2(aq)'), ('H2O(l)', 'HCHO(aq)', 'H2CO2(aq)'), ('CO(g)', 'CH3OH(aq)', 'CO2(g)'), ('CO(g)', 'CH3OH(aq)', 'HCHO(aq)'), ('